In [2]:
import scanpy as sc
#import anndata as AN
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from numba.typed import Dict
import numba as nb

from ABCA7lof2.singlecell import bulk_data, expressed_fraction


In [3]:
# load data
filtered_meta_final = np.load('./processed_data/single_cell/filtered_meta.npy', allow_pickle=True)
meta_names = np.load('./processed_data/single_cell/meta_names.npy', allow_pickle=True)
features = np.load('./processed_data/single_cell/features.npy')
logcounts = np.memmap('./processed_data/single_cell/filtered_logcounts.npy', mode='r+', shape=(filtered_meta_final.shape[0], features.shape[0]), dtype='float')
filtered_counts = np.memmap('./processed_data/single_cell/filtered_counts.npy', mode='r+', shape=(filtered_meta_final.shape[0], features.shape[0]), dtype='int16')

marker_indices = np.load('./processed_data/single_cell/marker_indices.npy')

In [4]:
annotations = np.load('./processed_data/single_cell/annotations.npy', allow_pickle=True)

In [5]:
# get pseudo-bulked data
index = filtered_meta_final[:,np.where(meta_names=='keep_cells_all_per_celltype')[0][0]].astype(bool)
index2 = filtered_meta_final[:,np.where(meta_names=='keep_cells_gaussian')[0][0]].astype(bool)
index_all = index&index2

In [6]:
len(annotations)==np.sum(index_all)

True

In [7]:
projids = filtered_meta_final[:,2]
celltype = annotations #filtered_meta_final[:,-3]
dictionary3 = dict(zip(np.unique(celltype), np.arange(len(np.unique(celltype)))))
celltype = [dictionary3[x] for x in celltype]

projids = projids.astype(int)
celltype = np.array(celltype).astype(int)

In [8]:
bulked = np.zeros(shape=(len(np.unique(projids))*len(np.unique(celltype)), features.shape[0]))
sums = np.zeros(len(np.unique(projids))*len(np.unique(celltype)))


dictionary1 = Dict(zip(np.unique(projids), np.arange(len(np.unique(projids)))))
dictionary2 = Dict(zip(np.unique(celltype), np.arange(len(np.unique(celltype))*len(np.unique(projids)), step=len(np.unique(projids)))))

exclude = set(np.where(np.invert(index_all))[0])

Summed, Ncells = bulk_data(logcounts, exclude, dictionary1, dictionary2, bulked, sums, projids, celltype)

/home/gridsan/djuna/.local/lib/python3.8/site-packages/numba/core/ir_utils.py:2149: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected set' found for argument 'exclude' of function 'bulk_data'.

For more information visit https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "ABCA7lof2/singlecell.py", line 5:
@nb.njit(parallel=True)
def bulk_data(logcounts, exclude, dictionary1, dictionary2, bulked, sums, projids, celltype):
^

  warnings.warn(NumbaPendingDeprecationWarning(msg, loc=loc))
/home/gridsan/djuna/.local/lib/python3.8/site-packages/numba/core/ir_utils.py:2149: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected set' found for argument 'exclude' of function '__numba_parfor_gufunc_0x7f56a7affee0'.

For more information visit https://numba.readthedocs.io/en/stable/reference/deprecation.htm

In [9]:
all_projids = np.tile(np.unique(projids), len(np.unique(celltype)))
all_celltype = np.repeat(np.unique(celltype), len(np.unique(projids)))

In [10]:





bulked_data = Summed/Ncells.reshape(-1,1)

np.save('./processed_data/single_cell/bulk_profiles.npy', bulked_data)
np.save('./processed_data/single_cell/bulk_profiles_rowIDs_1.npy', all_projids)
np.save('./processed_data/single_cell/bulk_profiles_rowIDs_2.npy', np.array(list(dictionary3.keys()))[all_celltype])


/state/partition1/slurm_tmp/23505678.0.0/ipykernel_53372/3200076833.py:1: RuntimeWarning: invalid value encountered in divide
  bulked_data = Summed/Ncells.reshape(-1,1)


In [11]:
# get pseudo-bulked data on counts
index = filtered_meta_final[:,np.where(meta_names=='keep_cells_all_per_celltype')[0][0]].astype(bool)
index2 = filtered_meta_final[:,np.where(meta_names=='keep_cells_gaussian')[0][0]].astype(bool)
index_all = index&index2

projids = filtered_meta_final[:,2]

celltype = annotations #filtered_meta_final[:,-3]
dictionary3 = dict(zip(np.unique(celltype), np.arange(len(np.unique(celltype)))))
celltype = [dictionary3[x] for x in celltype]

projids = projids.astype(int)
celltype = np.array(celltype).astype(int)

bulked = np.zeros(shape=(len(np.unique(projids))*len(np.unique(celltype)), features.shape[0]))
sums = np.zeros(len(np.unique(projids))*len(np.unique(celltype)))

dictionary1 = Dict(zip(np.unique(projids), np.arange(len(np.unique(projids)))))
dictionary2 = Dict(zip(np.unique(celltype), np.arange(len(np.unique(celltype))*len(np.unique(projids)), step=len(np.unique(projids)))))

exclude = set(np.where(np.invert(index_all))[0])

Summed, Ncells = bulk_data(filtered_counts, exclude, dictionary1, dictionary2, bulked, sums, projids, celltype)

all_projids = np.tile(np.unique(projids), len(np.unique(celltype)))
all_celltype = np.repeat(np.unique(celltype), len(np.unique(projids)))


np.save('./processed_data/single_cell/bulk_profiles_summed.npy', Summed)

/home/gridsan/djuna/.local/lib/python3.8/site-packages/numba/core/ir_utils.py:2149: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected set' found for argument 'exclude' of function 'bulk_data'.

For more information visit https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "ABCA7lof2/singlecell.py", line 5:
@nb.njit(parallel=True)
def bulk_data(logcounts, exclude, dictionary1, dictionary2, bulked, sums, projids, celltype):
^

  warnings.warn(NumbaPendingDeprecationWarning(msg, loc=loc))
/home/gridsan/djuna/.local/lib/python3.8/site-packages/numba/core/ir_utils.py:2149: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected set' found for argument 'exclude' of function '__numba_parfor_gufunc_0x7f56a7473e20'.

For more information visit https://numba.readthedocs.io/en/stable/reference/deprecation.htm

In [12]:
# get expressed genes per celltype
index = filtered_meta_final[:,np.where(meta_names=='keep_cells_all_per_celltype')[0][0]].astype(bool)
index2 = filtered_meta_final[:,np.where(meta_names=='keep_cells_gaussian')[0][0]].astype(bool)
index_all = index&index2

celltype = annotations
dictionary3 = dict(zip(np.unique(celltype), np.arange(len(np.unique(celltype)))))
celltype = [dictionary3[x] for x in celltype]

exclude = set(np.where(np.invert(index_all))[0])

expressed = np.zeros(shape=(len(np.unique(celltype)), len(features)))
summed = np.zeros(shape=len(np.unique(celltype)))
Ncells, expr = expressed_fraction(filtered_counts, exclude, celltype, expressed, summed)

np.save('./processed_data/single_cell/fraction_expressed.npy', expr/Ncells.reshape(-1,1))

/home/gridsan/djuna/.local/lib/python3.8/site-packages/numba/core/ir_utils.py:2149: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'celltype' of function 'expressed_fraction'.

For more information visit https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "ABCA7lof2/singlecell.py", line 16:
@nb.njit(parallel=True)
def expressed_fraction(counts, exclude, celltype, expressed, summed):
^

  warnings.warn(NumbaPendingDeprecationWarning(msg, loc=loc))
/home/gridsan/djuna/.local/lib/python3.8/site-packages/numba/core/ir_utils.py:2149: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected set' found for argument 'exclude' of function 'expressed_fraction'.

For more information visit https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-reflection-fo

In [15]:
exp = np.load('./processed_data/single_cell/fraction_expressed.npy')
meta = pd.read_csv('./raw_data/metadata/single_cell_individual_metadata.csv', index_col='projid')

In [18]:
gene_index = exp[dictionary3['Ast']]>0.1
cell_anno = np.load('./processed_data/single_cell/bulk_profiles_rowIDs_2.npy')

In [19]:
index = cell_anno=='Ast'
meta_sub = meta.loc[all_projids[index]]
counts_temp = pd.DataFrame(Summed[index][:,gene_index])
counts_temp.index = meta_sub.index
counts_temp.columns = features[gene_index][:,0]

In [21]:
from pydeseq2.dds import DeseqDataSet
from pydeseq2.ds import DeseqStats

meta_sub['ABCA7LoF'] = meta_sub['ABCA7LoF'].astype(object)
meta_sub['APOE4'] = meta_sub['apoe_genotype']>33

dds = DeseqDataSet(
    counts=counts_temp,
    clinical=meta_sub,
    design_factors=['seq_batch', 'ABCA7LoF', 'APOE4', 'msex', 'pmi'],  # compare samples based on the "condition"
    # column ("B" vs "A")
    refit_cooks=True,
    ref_level=['ABCA7LoF', '0'],
    n_cpus=8
)



/home/gridsan/djuna/.local/lib/python3.8/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [ ]:
dds.deseq2()


Fitting size factors...


/home/gridsan/djuna/.local/lib/python3.8/site-packages/pydeseq2/dds.py:310: RuntimeWarning: Every gene contains at least one zero, cannot compute log geometric means. Switching to iterative mode.
  self.fit_size_factors()


Fitting dispersions...
... done in 1.61 seconds.

Fitting MAP dispersions...
... done in 1.28 seconds.

Fitting dispersions...
... done in 2.33 seconds.

Fitting MAP dispersions...
... done in 1.37 seconds.

Fitting dispersions...
... done in 2.41 seconds.

Fitting MAP dispersions...
... done in 1.40 seconds.

Fitting dispersions...
... done in 2.37 seconds.

Fitting MAP dispersions...
... done in 1.33 seconds.



In [ ]:
stat_res = DeseqStats(dds, contrast=["disease", 'COVID-19', 'normal'], n_cpus=8)
